In [1]:
self = awg_spectrum

In [12]:
import pickle
from collections import OrderedDict

import numpy as np
import scipy.signal as sp

from core.configoption import ConfigOption
from core.module import Base
from core.util.modules import get_home_dir
from hardware.awg import SpectrumAWG35
from interface.microwave_interface import AWGLimits
from interface.pulser_interface import PulserInterface, PulserConstraints
from hardware.thirdparty.spectrum.pyspcm import *

In [322]:
def run_triggered_multi_from_list(seqs, memsize_seq=None):
        """
        seqs should have the 'waveform_ch1.pkl' form, i.e a list of waveform name saved on memory
        They should all be of equal length as far as I can understand. If not atleast the last should be the longest
        Single segment memory size determines how much is played after a trigger
        Eg: 
        ([
        ['sinA_a_ch0.pkl', 'sinA_a_ch1.pkl', 'sinA_d_ch1.pkl', 'sinA_d_ch1.pkl']
        ,['sinB_a_ch0.pkl', 'sinB_a_ch1.pkl', 'sinB_d_ch1.pkl', 'sinB_d_ch1.pkl']
        ],  'sinA_d_ch0.pkl')
        
        OR 
        
        ['sinA', 'sinB']
        """
        self.instance.set_mode('multi')       
        sample_rate = self.get_sample_rate()
        self.set_reps(0)
        path = self.waveform_folder
        wave_form_list = self.get_waveform_names()
        
        # these are .pkl file names from saved_pulsed_asseets which simply have everything that happens in a channel during the
        # entire ensmeble. Multiple ensembles can be sequenced into memory to play one after the other on receiving the trigger.
        def wfm_l_maker(ensemble_names=[]):
            wfm_l = []
            for iens, ens in enumerate(ensemble_names):
                l = [f'{ens}_a_ch{i}.pkl' for i in range(5)]
                wfm_l.append(l)
                l = [f'{ens}_d_ch{i}.pkl' for i in range(6)]
                wfm_l[iens].extend(l)
            return wfm_l
        
        if isinstance(seqs[0], str):
            seqs = wfm_l_maker(seqs)            
            
        # load ensemble to determine the sequence size. This will determine how much is played after one external
        # trigger. It is assumed that all the ensembles are of equal length.
        waveform = seqs[-1][-1] if memsize_seq is None else memsize_seq
        load_dict = dict()
        wave_name = waveform.rsplit('.pkl')[0]
        channel_num = int(wave_name.rsplit('_ch', 1)[1])
        # Map channel numbers to HW channel numbers
        if '_a_ch' not in waveform:
            channel = channel_num + 4
        else:
            channel = channel_num
        load_dict[channel] = wave_name

        if not load_dict:
            self.log.error('No data to send to AWG')
            return -1

        for ch, value in load_dict.items():
            if value in wave_form_list:
                wavefile = '{0}.pkl'.format(value)
                filepath = os.path.join(path, wavefile)
                data = self.my_load_dict(filepath)
            else:
                self.log.error('Cannot find waveform to send to AWG')
                return -1
        
        max_seq = data
        segment_size = int(len(max_seq))       
        while not segment_size % 32 == 0:
            segment_size += 1

        self.instance.set_segment_size(segment_size)
        self.instance.set_memory_size(segment_size * len(seqs))
        self.instance.init_ext_trigger()
        
        # setting of seqment size,i.e, replay length, and init of trigger done.
        
        # loading of the ensemble data, separating into channel .pkl files and then writing into data list done here
        # after data_list for entire ensemble is compiled, upload is done into AWG specifying the segment size
        # and also the index this particular segment occupies in memory.
        for iseq, seq in enumerate(seqs):
            data_list = list()
            for i in range(4):
                data_list.append(np.zeros(int(segment_size), np.int16))
            for i in range(6):
                data_list.append(np.zeros(int(segment_size), np.bool))
                
            load_dict = seq
            new_dict = dict()
            for waveform in load_dict:
                wave_name = waveform.rsplit('.pkl')[0]
                channel_num = int(wave_name.rsplit('_ch', 1)[1])
                if '_a_ch' not in waveform:
                    channel = channel_num + 4
                else:
                    channel = channel_num
                new_dict[channel] = wave_name
            load_dict = new_dict

            if not load_dict:
                self.log.error('No data to send to AWG')
                return -1

            for ch, value in load_dict.items():
                if value in wave_form_list:
                    wavefile = '{0}.pkl'.format(value)
                    filepath = os.path.join(path, wavefile)
                    data = self.my_load_dict(filepath)
                    data_list[ch] = data
                    data_size = len(data)

                    if '_a_ch' in value:
                        chan_name = 'a_ch{0}'.format(value.rsplit('a_ch')[1])
                        self.loaded_assets[chan_name] = value[:-6]
                    else:
                        chan_name = 'd_ch{0}'.format(value.rsplit('d_ch')[1])
                        self.loaded_assets[chan_name] = value[:-6]
                else:
                    self.log.warn('Waveform {} not found in {}'.format(value, self.waveform_folder))
                    data_size = 0

            # See pg 130 in manual
            count = 0
            while not data_size % 32 == 0:
                data_size += 1
                count += 1
            if not count == 1:
                extra = np.zeros(count, np.int16)
                new_list = list()
                for row in data_list:
                    new_row = np.concatenate((row, extra), axis=0)
                    new_list.append(new_row)
                data_list = new_list

            self.log.info(f'Uploading waveform set {seq} to AWG...')
            if not data_size == 0:
                self.instance.upload(data_list, segment_size, segment_size * iseq)
                self.typeloaded = 'waveform'
        self.log.info('Upload to AWG complete')

        del seqs

In [323]:
self.pulser_off()
run_triggered_multi_from_list(['sinA', 'sinB'])
self.pulser_on()

here
[['sinA_a_ch0.pkl', 'sinA_a_ch1.pkl', 'sinA_a_ch2.pkl', 'sinA_a_ch3.pkl', 'sinA_a_ch4.pkl', 'sinA_d_ch0.pkl', 'sinA_d_ch1.pkl', 'sinA_d_ch2.pkl', 'sinA_d_ch3.pkl', 'sinA_d_ch4.pkl', 'sinA_d_ch5.pkl'], ['sinB_a_ch0.pkl', 'sinB_a_ch1.pkl', 'sinB_a_ch2.pkl', 'sinB_a_ch3.pkl', 'sinB_a_ch4.pkl', 'sinB_d_ch0.pkl', 'sinB_d_ch1.pkl', 'sinB_d_ch2.pkl', 'sinB_d_ch3.pkl', 'sinB_d_ch4.pkl', 'sinB_d_ch5.pkl']]


1

In [266]:
self.pulser_off()

1

In [ ]:
# wfm_l = (['sinA_d_ch0.pkl', 'sinA_d_ch1.pkl',
#          'sinB_d_ch0.pkl', 'sinB_d_ch1.pkl'],  'sinA_d_ch0.pkl')